In [30]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [31]:
# read the data file:
df = pd.read_csv('houses.csv')

In [32]:
# show the first 5 rows of the data:
df.head()

,URL,State,Property_type,Date,Ort,Title,Kaufpreis,Courtage fÃ¼r KÃ¤ufer,Zimmer,Anzahl ParkflÃ¤chen,...,GrundstÃ¼cksflÃ¤che ca.,Zustand,Baujahr,VerfÃ¼gbar ab,Energieeffizienzklasse,Endenergiebedarf,Energieausweis,Heizungsart,Befeuerungsart,Baujahr (laut Energieausweis)
0,https://www.immonet.de/angebot/48554521,Schleswig-Holstein,Haus,2022-09-29 15:39:02.285509,LutherstraÃe 20-22--24837-Schleswig,Wohntraum an der Schlei,"475,900Â",Provisionsfrei,5.0,1.0,...,132,Erstbezug,NaN,NaN,Klasse A,43.3 kWh/(mÂ²*a),Energiebedarfsausweis,NaN,FernwÃ¤rme,NaN
1,https://www.immonet.de/angebot/48261463,Schleswig-Holstein,Haus,2022-09-29 15:39:02.339250,22926-Ahrensburg,"LichterfÃ¼llte, stylische Bauhausstil-Villa in...","1,895,000Â","3,57 % inkl. MwSt.",6.0,NaN,...,"1,011",Neuwertig,2001.0,nach Vereinbarung,NaN,151.32 kWh/(mÂ²*a),Energiebedarfsausweis,Zentralheizung,Gas,2001.0
2,https://www.immonet.de/angebot/48602242,Schleswig-Holstein,Haus,2022-09-29 15:39:02.385231,23569-LÃ¼beck,Handwerker aufgepasst- kleine DoppelhaushÃ¤lft...,"219,000Â","3,33% inkl. MwSt.",3.5,NaN,...,751,Altbau,1937.0,nach Vereinbarung,Klasse H,282.8 kWh/(mÂ²*a),Energiebedarfsausweis,Zentralheizung,Gas,1937.0
3,https://www.immonet.de/angebot/48579235,Schleswig-Holstein,Haus,2022-09-29 15:39:02.415756,24619-RendswÃ¼hren,Ein Einfamilienhaus fÃ¼r die gesamte Familie m...,"499,000Â",keine besondere Angabe,6.0,6.0,...,"1,136",NaN,1977.0,NaN,Klasse E,152.9 kWh/(mÂ²*a),Energiebedarfsausweis,Zentralheizung,Ãl,1977.0
4,https://www.immonet.de/angebot/47991507,Schleswig-Holstein,Haus,2022-09-29 15:39:02.465178,25451-Quickborn,Ein StÃ¼ck Sylt in Quickborn-Heide - Einzigart...,"1,590,000Â","3,20 % v. Netto-VKPreis (inkl. MwSt)",5.0,NaN,...,"1,074",NaN,1962.0,nach Vereinbarung,Klasse H,269.5 kWh/(mÂ²*a),Energieverbrauchsausweis,NaN,Gas,2020.0


In [33]:
# conver the numeric columns to float type:

df['Kaufpreis'] = df['Kaufpreis'].str.replace('Â\xa0','').str.replace(',','').astype('float')

df['WohnflÃ¤che ca.'] = df['WohnflÃ¤che ca.'].str.replace(',','').astype('float')

df['GrundstÃ¼cksflÃ¤che ca.'] = df['GrundstÃ¼cksflÃ¤che ca.'].str.replace(',','').astype('float')

# remove gaps:
df['Befeuerungsart'] = df['Befeuerungsart'].str.replace('\t','').str.replace('\n','')

# replace values under 1000 in Baujahr column with nan:

df['Baujahr'] = df['Baujahr'].apply(lambda x :np.nan if x < 1000 else x)

# make all values in Grund column in range (10, 500):

df['GrundstÃ¼cksflÃ¤che ca.'] = df['GrundstÃ¼cksflÃ¤che ca.'].apply(lambda x :np.nan if (x < 10 or x > 5000) else x)


# fill missing values in Befeuerungsart column:

# df['Befeuerungsart'] = df['Befeuerungsart'].fillna(df['Befeuerungsart'].mode().iloc[0])


In [34]:
# extract the Postleitzahl from Ort column to a new column:

df['Postleitzahl'] = df['Ort'].str.extract('(\d{5})').astype('str')

In [35]:
# remove unused columns:

df = df.drop(columns=['Property_type','URL', 'State','Title','Date','Heizungsart', 'Energieausweis', 'Endenergiebedarf', 'Energieeffizienzklasse', 'VerfÃ¼gbar ab', 'Anzahl ParkflÃ¤chen', 'Baujahr (laut Energieausweis)', 'Ort','Courtage fÃ¼r KÃ¤ufer','Zustand'])

In [36]:
# calculate the meter price and add it to a new cloumn

df['M_Kaufpreis'] = df['Kaufpreis'] / df['WohnflÃ¤che ca.']

In [37]:
# outliers:
df = df.loc[df['Kaufpreis'] < 5000000]
df = df.loc[df['Zimmer'] < 20]
df = df.loc[df['Baujahr'] > 1650]
# #df = df.loc[df['M_Kaufpreis'] < 15000]


In [38]:
# calculate the meter price for each area and add it in new cloumn "AreaCode":

def getOrtMean(ort):
  return df.loc[df['Postleitzahl'] == ort]['M_Kaufpreis'].mean()

Orts = []
for ort in df['Postleitzahl'].unique():
  Orts.append([ort, getOrtMean(ort)])

def getOrtCode(ort):
  ind = -1
  for i , sublist in enumerate(Orts):
    if ort in sublist:
      ind = i 
  return Orts[ind][1]

df['AreaCode'] = df['Postleitzahl'].apply(getOrtCode)


In [39]:
# we don't need the "Postleitzahl" and "M_Kaufpreis" any more so we will drop it :

df = df.drop(columns=['Postleitzahl','M_Kaufpreis'])

In [40]:
# show the data info:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46175 entries, 1 to 55238
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Kaufpreis                46175 non-null  float64
 1   Zimmer                   46175 non-null  float64
 2   WohnflÃ¤che ca.          46175 non-null  float64
 3   GrundstÃ¼cksflÃ¤che ca.  44604 non-null  float64
 4   Baujahr                  46175 non-null  float64
 5   Befeuerungsart           40359 non-null  object 
 6   AreaCode                 46175 non-null  float64
dtypes: float64(6), object(1)
memory usage: 2.8+ MB


In [41]:
# Use KNN imputer to impute the missing values in Baujahr and Grund columns:

imputer = KNNImputer()
df['Baujahr'] = imputer.fit_transform(df[['Baujahr']])
df['GrundstÃ¼cksflÃ¤che ca.'] = imputer.fit_transform(df[['GrundstÃ¼cksflÃ¤che ca.']])

In [42]:
# show some statistical info about the data:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Kaufpreis,46175.0,565451.115593,446134.306033,0.010000,300000.000000,455000.000000,679844.000000,4999000.0
Zimmer,46175.0,6.371705,2.840459,1.000000,4.500000,6.000000,7.500000,19.5
WohnflÃ¤che ca.,46175.0,190.323787,1140.093524,1.000000,123.390000,154.000000,210.000000,165000.0
GrundstÃ¼cksflÃ¤che ca.,46175.0,745.259120,616.680733,10.000000,365.000000,620.000000,881.000000,5000.0
Baujahr,46175.0,1966.560260,46.643172,1658.000000,1946.000000,1972.000000,2000.000000,2025.0
AreaCode,46175.0,3427.546567,5692.215098,214.723926,2198.509042,2926.564297,4030.455219,876000.0


In [43]:
# split the data:
X, y = df.drop("Kaufpreis",axis=1) , df["Kaufpreis"]

# encode the data using one hot encoder :
encoder= OneHotEncoder()
X = encoder.fit_transform(X)

# scaling (in case we want to use a distance-based algorithm)
 
# scaler = StandardScaler(with_mean=False)
# X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# visualize the change of the mean absolute error according to the change of parameters: 

def max_depth_mae(max_depth):
    model= DecisionTreeRegressor(max_depth=max_depth,random_state=0)
    model.fit(X_train,y_train)
    return mean_absolute_error(y_test,model.predict(X_test))

def min_samples_split_mae(min_samples_split):
    model=DecisionTreeRegressor(min_samples_split=min_samples_split,random_state=0)
    model.fit(X_train,y_train)
    return mean_absolute_error(y_test,model.predict(X_test))

## How a decission tree will perform with this range of max_depth values
max_depth=[x for x in range(20,150,2)]
md_mae=[]
for md in max_depth:
    md_mae.append(max_depth_mae(md))
fig = px.line(y=md_mae,x=max_depth, title='mean absolut error _ maxdepth',width=600, height=400)
fig.show()

mss_mae=[]
mss=[x for x in range(2,120,4)]
for min_samples_split in mss:
    mss_mae.append(min_samples_split_mae(min_samples_split))
fig = px.line(y=mss_mae,x= mss, title='mean absolut error_ min_samples_split',width=600, height=400)
fig.show()

In [45]:
# Decision Tree Regressor()
dtr = DecisionTreeRegressor(min_samples_split=38)
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
print(f' the r2 score: {r2_score(y_test, y_pred)}')
print(f'the mean absolute error: {mean_absolute_error(y_test, y_pred)}')
print(f' the man absolute percentage error: {mean_absolute_percentage_error(y_test, y_pred)}')

 the r2 score: 0.09691999213921165
the mean absolute error: 241360.88899516466
 the man absolute percentage error: 0.5679314543047683


In [46]:
# plot the predicted values comparing to the actual values:

fig = px.scatter(x = y_test, y= y_pred)
fig.show()